In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#show all columns and rows
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
from datetime import datetime, timedelta

In [3]:
import requests
import json

# weather - darksky data 2015-2019
### data from API retrieved https://darksky.net/dev/docs#time-machine-request

In [4]:
#key = # generate your own https://darksky.net/dev

In [273]:
#date = datetime.strptime('2015-01-01', '%Y-%m-%d')
#timest = int(datetime.timestamp(date))

In [ ]:
#data_darksky = []
#for i in range(365):
#    darksky = json.loads(requests.get('https://api.darksky.net/forecast/{}/52.2193,21.0047,{}'.format(key, timest)).text)
#    data_darksky.extend(darksky['hourly']['data'])
#    timest += 24*60*60
#    
#df_darksky = pd.DataFrame(dane_darksky) 

In [4]:
# ready data from a file
d56789 = pd.read_csv('data/data_darksky_hourly_56789.csv',index_col = 0, header = 0)
d56789.shape

(43847, 19)

In [5]:
d56789['date'] = d56789['time'].apply(lambda x: datetime.fromtimestamp(x))

In [6]:
d56789.dtypes

apparentTemperature           float64
cloudCover                    float64
dewPoint                      float64
humidity                      float64
icon                           object
ozone                         float64
precipAccumulation            float64
precipIntensity               float64
precipProbability             float64
precipType                     object
pressure                      float64
summary                        object
temperature                   float64
time                            int64
uvIndex                       float64
visibility                    float64
windBearing                   float64
windGust                      float64
windSpeed                     float64
date                   datetime64[ns]
dtype: object

# pollution PM2.5 - GIOŚ data 2015-2019
### from archive http://powietrze.gios.gov.pl/pjp/archives

In [7]:
df_pm25 = pd.read_csv('data/data_pm25_2015-9.csv',index_col = 0, header = 0)

In [8]:
df_pm25['date']= pd.to_datetime(df_pm25['date'], format = '%Y-%m-%d %H:%M:%S')
print(df_pm25.dtypes)
df_pm25.tail()

pm25_nie           float64
pm25_kon           float64
pm25_wok           float64
date        datetime64[ns]
dtype: object


,pm25_nie,pm25_kon,pm25_wok,date
8754,12.4,6.8,6.4,2019-12-31 20:00:00
8755,13.5,4.4,6.6,2019-12-31 21:00:00
8756,10.8,4.0,5.1,2019-12-31 22:00:00
8757,10.3,6.1,5.3,2019-12-31 23:00:00
8758,11.8,10.2,6.7,2020-01-01 00:00:00


# list of dates 2015-2019

In [9]:
list_date = []

In [10]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)*24+24):
        yield date1 + timedelta(n/24)

In [11]:
start_dt = datetime(2015, 1, 1)
end_dt = datetime(2019, 12, 31)

for dt in daterange(start_dt, end_dt):
    list_date.append(dt.strftime("%Y-%m-%d %H:%M:%S"))

In [12]:
len(list_date), list_date[0], list_date[-1]

(43824, '2015-01-01 00:00:00', '2019-12-31 23:00:00')

In [13]:
df = pd.DataFrame(data = {'date': list_date})
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d %H:%M:%S')
print(df.dtypes)
df.head()

date    datetime64[ns]
dtype: object


,date
0,2015-01-01 00:00:00
1,2015-01-01 01:00:00
2,2015-01-01 02:00:00
3,2015-01-01 03:00:00
4,2015-01-01 04:00:00


# merge all data

In [14]:
df_all = pd.merge( df, df_pm25, on=['date'], how='left')
df_all = pd.merge( df_all, d56789, on=['date'], how='left')
df_all.tail()

,date,pm25_nie,pm25_kon,pm25_wok,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
43848,2019-12-31 19:00:00,11.6,6.9,7.0,34.07,0.82,34.28,0.75,partly-cloudy-night,335.4,NaN,0.0,0.0,NaN,1022.5,Mostly Cloudy,41.51,1.577815e+09,0.0,10.0,307.0,28.66,14.02
43849,2019-12-31 20:00:00,12.4,6.8,6.4,33.26,0.59,34.00,0.76,partly-cloudy-night,336.1,NaN,0.0,0.0,NaN,1023.3,Partly Cloudy,40.97,1.577819e+09,0.0,10.0,307.0,27.30,14.39
43850,2019-12-31 21:00:00,13.5,4.4,6.6,32.30,0.29,33.20,0.76,clear-night,337.8,NaN,0.0,0.0,NaN,1024.3,Clear,40.22,1.577822e+09,0.0,10.0,305.0,26.36,14.43
43851,2019-12-31 22:00:00,10.8,4.0,5.1,31.28,0.05,32.37,0.76,clear-night,338.0,NaN,0.0,0.0,NaN,1025.1,Clear,39.35,1.577826e+09,0.0,10.0,303.0,25.92,14.14
43852,2019-12-31 23:00:00,10.3,6.1,5.3,30.33,0.02,31.74,0.76,clear-night,334.5,NaN,0.0,0.0,NaN,1025.7,Clear,38.58,1.577830e+09,0.0,10.0,303.0,26.24,14.04


# filling missing data

In [15]:
df_all.isnull().sum()

date                       0
pm25_nie                 568
pm25_kon                9860
pm25_wok                1290
apparentTemperature        6
cloudCover              1417
dewPoint                   6
humidity                   6
icon                    1634
ozone                  30340
precipAccumulation     43130
precipIntensity          681
precipProbability        681
precipType             38411
pressure                 933
summary                 1634
temperature                6
time                       6
uvIndex                  583
visibility                13
windBearing              291
windGust               10583
windSpeed                  9
dtype: int64

In [16]:
df_all.columns

Index(['date', 'pm25_nie', 'pm25_kon', 'pm25_wok', 'apparentTemperature',
       'cloudCover', 'dewPoint', 'humidity', 'icon', 'ozone',
       'precipAccumulation', 'precipIntensity', 'precipProbability',
       'precipType', 'pressure', 'summary', 'temperature', 'time', 'uvIndex',
       'visibility', 'windBearing', 'windGust', 'windSpeed'],
      dtype='object')

In [17]:
# interpolate

cols=[ 'pm25_nie', 'pm25_kon', 'pm25_wok',
       'apparentTemperature', 'cloudCover', 'dewPoint',
       'humidity',  'precipType', 'pressure', 'temperature',
       'time', 'uvIndex', 'visibility', 'windBearing',
       'windSpeed']

for col in cols:
    df_all[col] = df_all[col].interpolate(limit_direction='both')

In [18]:
# fill with 0

cols=['precipAccumulation','precipIntensity','precipProbability']

for col in cols:
    df_all[col] = df_all[col].fillna(0)

In [19]:
# value from another column

df_all['windGust'] = df_all['windGust'].fillna(df_all['windSpeed'])

In [ ]:
#dict_fac_summary = pd.factorize(d789['summary'])
#d789['fac_summary'] =pd.factorize(d789['summary'])[0]

#dict_fac_precipType = pd.factorize(d789['precipType'])
#d789['fac_precipType'] =pd.factorize(d789['precipType'])[0]

In [20]:
df_all.isnull().sum()

date                       0
pm25_nie                   0
pm25_kon                   0
pm25_wok                   0
apparentTemperature        0
cloudCover                 0
dewPoint                   0
humidity                   0
icon                    1634
ozone                  30340
precipAccumulation         0
precipIntensity            0
precipProbability          0
precipType             38411
pressure                   0
summary                 1634
temperature                0
time                       0
uvIndex                    0
visibility                 0
windBearing                0
windGust                   0
windSpeed                  0
dtype: int64

In [21]:
df_all.dtypes

date                   datetime64[ns]
pm25_nie                      float64
pm25_kon                      float64
pm25_wok                      float64
apparentTemperature           float64
cloudCover                    float64
dewPoint                      float64
humidity                      float64
icon                           object
ozone                         float64
precipAccumulation            float64
precipIntensity               float64
precipProbability             float64
precipType                     object
pressure                      float64
summary                        object
temperature                   float64
time                          float64
uvIndex                       float64
visibility                    float64
windBearing                   float64
windGust                      float64
windSpeed                     float64
dtype: object

# additional variables

### time

In [22]:
df_all['year'] = df_all['date'].apply(lambda x: x.year)
df_all['month'] = df_all['date'].apply(lambda x: x.month)
df_all['day'] = df_all['date'].apply(lambda x: x.day)
df_all['hour'] = df_all['date'].apply(lambda x: x.hour)
df_all['day_of_week'] = df_all['date'].apply(lambda x: x.isocalendar()[2])
df_all['no_of_week'] = df_all['date'].apply(lambda x: x.isocalendar()[1])

df_all.head()       

,date,pm25_nie,pm25_kon,pm25_wok,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,year,month,day,hour,day_of_week,no_of_week
0,2015-01-01 00:00:00,51.5034,78.085,51.320000,32.93,1.0,29.56,0.87,NaN,NaN,0.0,0.0,0.0,NaN,1027.7,NaN,32.93,1.420067e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,0,4,1
1,2015-01-01 01:00:00,71.8204,78.085,68.982316,33.90,1.0,31.11,0.89,NaN,NaN,0.0,0.0,0.0,NaN,1027.7,NaN,33.90,1.420070e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,1,4,1
2,2015-01-01 02:00:00,42.6996,64.460,48.707108,27.73,1.0,31.59,0.91,cloudy,NaN,0.0,0.0,0.0,NaN,1027.7,Overcast,33.81,1.420074e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,2,4,1
3,2015-01-01 03:00:00,38.2824,36.210,37.986883,27.73,1.0,32.76,0.96,fog,NaN,0.0,0.0,0.0,NaN,1027.7,Foggy,33.81,1.420078e+09,0.0,1.244,251.0,6.93,6.93,2015,1,1,3,4,1
4,2015-01-01 04:00:00,35.4194,29.585,33.675489,26.41,1.0,32.89,0.97,fog,NaN,0.0,0.0,0.0,NaN,1027.7,Foggy,33.73,1.420081e+09,0.0,1.152,251.0,8.96,8.96,2015,1,1,4,4,1


### winter break

In [23]:
df_all['winter_break'] = 0

In [24]:
mask = (df_all['date'] >='2015-01-19') & (df_all['date'] <='2015-02-01')
df_all['winter_break'][mask] = 1

In [25]:
mask = (df_all['date'] >='2016-02-01') & (df_all['date'] <='2016-02-12')
df_all['winter_break'][mask] = 1

In [26]:
mask = (df_all['date'] >='2017-02-13') & (df_all['date'] <='2017-02-26')
df_all['winter_break'][mask] = 1

In [27]:
mask = (df_all['date'] >='2018-01-15') & (df_all['date'] <='2018-01-28')
df_all['winter_break'][mask] = 1

In [28]:
mask = (df_all['date'] >='2019-01-28') & (df_all['date'] <='2019-02-10')
df_all['winter_break'][mask] = 1

# saving data for futher analysis

In [29]:
df_all.to_csv('data/data_pm25_plus_darksky.csv')